In [6]:
import os 
import json
import time
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cbook as cbook
#from scipy.misc import imread
from matplotlib.pyplot import imread

from importlib import reload
%load_ext autoreload
%autoreload 2
import AnalyzeAnnotations as aa
import PlotAnnotations as pa

pd.options.display.max_colwidth = 400
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### <span style="color:red"> Load sloth jsons </span> into array of pandas dataframes

In [64]:
# load all json files from datapath into array of pandas dataframes
json_path = './data/testdata_12_22/'


print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}:      Path to be loaded: {json_path}")
df_stats, all_df, json_files, all_imgnames = aa.loadAllJSONSFromPath(json_path)
number_jsons = len([file for file in os.listdir(imgs_path) if os.path.isfile(os.path.join(imgs_path, file))])
print(f'Number of json files loaded: {number_jsons}')
print(f'Number of annotated images loaded: {len(all_imgnames)}')

28/12/2022, 21:20:59:      Path to be loaded: ./data/testdata_12_22/
Number of json files to load: 5
Loading: ./data/testdata_12_22/Batch0019_0026_after_analysis.json
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0662.JPG
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0675.JPG
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0711.JPG
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0720.JPG
Loading: ./data/testdata_12_22/Batch0019_0026_before_analysis.json
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0592.JPG
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0595.JPG
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0625.JPG
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0636.JPG
Loading: ./data/testdata_12_22/Batch0027_0028_before_analysis.json
..\batch0027-0028_d

#### Calculate abundance of classes + split positions + average pos column + length column + direction (rad)

In [65]:
# update df_stats with abundance of classes
df_stats = aa.getNumberOfClassesInDFs(df_stats, all_df)
# extract head and tail positions + calculate average of head and tail + calc length + calc polarization
df_allsplit = aa.splitPos(df_stats, all_df, all_imgnames)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_stats)
print(f"\n{df_allsplit[0][1].head(5)}")

                                                                                           Ntags_f1  \
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0662.JPG              18   
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0675.JPG               0   
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0711.JPG               9   
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0720.JPG              21   
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0592.JPG             9   
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0595.JPG            14   
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0625.JPG             8   
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0636.JPG            16   
..\batch0027-0028_densitybefore_Canon400D\Batch0027_0028_before_analysis\

#### Stats of dataframe df_stats

In [45]:
#df.describe()
df_stats.describe()

,Ntags_f1,Ntags_f2,Ntags_f3,Ntags_total
count,12.000000,12.000000,12.000000,12.000000
mean,10.333333,3.750000,14.000000,28.083333
std,6.429101,3.840573,13.114877,18.272972
min,0.000000,0.000000,0.000000,0.000000
25%,7.750000,0.000000,7.750000,23.250000
50%,9.500000,3.000000,9.000000,27.500000
75%,14.500000,6.250000,19.000000,31.000000
max,21.000000,10.000000,46.000000,65.000000


#### Neighbours

* Calculate distance matrix and find neighbors
 * finds neighbors using a distance matrix and filtering out neighbours over two times average length and four times average length
* Add number of class neighbors column
 * adds columns for each fish containing the number of neighbors of each class (for distance d2/av2 and d4/av4)
* Add same class neighbors percentage and average number of neighbors (per class and total) to stats 
* degree: mean number of fish in radius of 2 body lengths
* density: degree / number of fish in image
* nnd: euclidean distances to nearest neighbor

In [66]:
df_allsplit, df_stats = aa.neighbor_calculations(df_allsplit, df_stats, json_files, all_imgnames)
# 
# check sample
#

pd.set_option("display.max_rows", None, "display.max_columns", None)

print(df_allsplit[4][1].head(2))
df_stats.head(3)

..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0662.JPG:   total avg 2BL: 136.01685 , total avg 4BL: 272.03369, f1 avg 2BL: 167.81249, f2 avg 2BL: 278.86914 
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0711.JPG:   total avg 2BL: 180.76429 , total avg 4BL: 361.52857, f1 avg 2BL: 193.14618, f2 avg 2BL: 229.01041 
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0720.JPG:   total avg 2BL: 139.17782 , total avg 4BL: 278.35564, f1 avg 2BL: 155.65401, f2 avg 2BL: 178.73431 
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0592.JPG:   total avg 2BL: 125.60284 , total avg 4BL: 251.20567, f1 avg 2BL: 135.30094, f2 avg 2BL: nan 
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0595.JPG:   total avg 2BL: 196.37252 , total avg 4BL: 392.74503, f1 avg 2BL: 206.52532, f2 avg 2BL: nan 
..\batch0019-0026_densitybefore_Canon400D\Batch0019_0026_before_analysis\IMG_0

,Ntags_f1,Ntags_f2,Ntags_f3,Ntags_total,meanNND_total,meanBL_px_f1,meanBL_px_f2,meanBL_px_f3,PercSameSpecNeighbors_per2BL_f1,PercSameSpecNeighbors_per2BL_f2,PercSameSpecNeighbors_per2BL_f3,NSameSpecNeighbors_per2BL_total,NSameSpecNeighbors_per2BL_f1,NSameSpecNeighbors_per2BL_f2,NSameSpecNeighbors_per2BL_f3,PercSameSpecNeighbors_per2f1BL_f1,PercSameSpecNeighbors_per2f2BL_f2,NSameSpecNeighbors_per2f1BL_f1,NSameSpecNeighbors_per2f2BL_f2,meanDegree_per2BL_total,meanDensity_per2BL_total,PercSameSpecNeighbors_per4BL_f1,PercSameSpecNeighbors_per4BL_f2,PercSameSpecNeighbors_per4BL_f3,NSameSpecNeighbors_per4BL_total,NSameSpecNeighbors_per4BL_f1,NSameSpecNeighbors_per4BL_f2,NSameSpecNeighbors_per4BL_f3
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0662.JPG,18,1,46,65,154.751814,83.906244,139.434572,60.234795,0.305556,0.0,0.373188,0.923077,0.333333,0.0,0.695652,0.37037,0.0,0.555556,0.0,0.923077,0.014201,0.519048,0.0,0.720445,3.600000,2.111111,0.0,2.73913
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0675.JPG,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0711.JPG,9,4,8,21,370.015705,96.573090,114.505206,71.355795,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.555556,0.0,0.250000,0.761905,0.888889,0.0,0.25000


### Add average length, direction and polarization to stats

In [67]:
df_stats = aa.pop_stats_pol_dir_len(df_allsplit, df_stats)

df_stats.head(3)

,Ntags_f1,Ntags_f2,Ntags_f3,Ntags_total,meanNND_total,meanBL_px_f1,meanBL_px_f2,meanBL_px_f3,PercSameSpecNeighbors_per2BL_f1,PercSameSpecNeighbors_per2BL_f2,PercSameSpecNeighbors_per2BL_f3,NSameSpecNeighbors_per2BL_total,NSameSpecNeighbors_per2BL_f1,NSameSpecNeighbors_per2BL_f2,NSameSpecNeighbors_per2BL_f3,PercSameSpecNeighbors_per2f1BL_f1,PercSameSpecNeighbors_per2f2BL_f2,NSameSpecNeighbors_per2f1BL_f1,NSameSpecNeighbors_per2f2BL_f2,meanDegree_per2BL_total,meanDensity_per2BL_total,PercSameSpecNeighbors_per4BL_f1,PercSameSpecNeighbors_per4BL_f2,PercSameSpecNeighbors_per4BL_f3,NSameSpecNeighbors_per4BL_total,NSameSpecNeighbors_per4BL_f1,NSameSpecNeighbors_per4BL_f2,NSameSpecNeighbors_per4BL_f3,meanBL_px_total,meanDirection_rad_total,meanPolarization_total,meanPolarizationError_total
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0662.JPG,18,1,46,65,154.751814,83.906244,139.434572,60.234795,0.305556,0.0,0.373188,0.923077,0.333333,0.0,0.695652,0.37037,0.0,0.555556,0.0,0.923077,0.014201,0.519048,0.0,0.720445,3.600000,2.111111,0.0,2.73913,68.008423,0.567782,0.068744,0.124035
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0675.JPG,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
..\batch0019-0026_densityafter_Canon400D\Batch0019_0026_after_analysis\IMG_0711.JPG,9,4,8,21,370.015705,96.573090,114.505206,71.355795,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.555556,0.0,0.250000,0.761905,0.888889,0.0,0.25000,90.382143,2.312602,0.728509,0.218218


#### Export to csv

In [74]:
if not os.path.exists('./output/stats'):
            os.makedirs('./output/stats')

for i, df_split in enumerate(df_allsplit):
    output_name = df_split[0][3:].replace('\\','_')
    df_split[1].to_csv('./output/stats/' + output_name + '.csv')

df_stats.to_csv('./output/summary.csv')

### Plot positions

In [ ]:
#test one image
index=4
#print(json_files)
number = df_allsplit[index].number
imgpath = pa.get_imgpath_for_number(number, imgs_path)
pa.plot_pos_cat_img(df_allsplit[index], imgpath, show=True, save=True)
pa.plot_pos_ori_cat(df_allsplit[index], show=True, save=True)
pa.plot_pos_ori_cat_img(df_allsplit[index], imgpath, show=True, save=True)

In [ ]:
# Plot positions of all read files
pa.plot(df_allsplit, json_files, imgs_path, SHOW=False, SAVE=True)

print(f"{time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime())}")

## TODO

* refactor much
* convert .ipynb to .py
* create exe file from python script